In [3]:
#!conda install -y -c conda-forge pyaudio
#!sudo apt install -y libasound2-plugins-extra
#!conda install -y -c conda-forge librosa

In [4]:
import requests
import os, time, sys, subprocess
import json
from IPython.display import JSON

In [88]:
vowel_map = {
    "a": (0, 1),
    "i": (0.25, 0.5),
    "e": (0.5, 0.8),
    "o": (0.75, 0.75),
    "u": (1.0, 0.7)
}
consonant_map = {
    "k": (0.25, 0.25),
    "s": (0.5, 0.3),
    "t": (0.5, 0.5),
    "n": (0.5, 0.5),
    "h": (None, 0.6),
    "m": (None, 0),
    "y": (0.75, 0.75),
    "w": (0.75, 0.25),
    "N": (0.5, 0)
}

def update(w, t1, t2):
    mouth_map = {}
    if w in consonant_map:
        if consonant_map[w][0]:
            mouth_map["mouth_x"] = consonant_map[w][0]
        if consonant_map[w][1]:
            mouth_map["mouth_y"] = consonant_map[w][1]
    if w in vowel_map:
        mouth_map = {}
        if vowel_map[w][0]:
            mouth_map["mouth_x"] = vowel_map[w][0]
        if vowel_map[w][1]:
            mouth_map["mouth_y"] = vowel_map[w][1]

    # Sending WebHook request to Inochi2D.
    # Currently, requests.post disconnected from WebHook adapter everytime after sending action.
    # Should develop non-blocking, keep connection method.
    while time.time() - t1 < t2:
        try:
            res = requests.post("http://localhost:9999/blendshapes", json=mouth_map, timeout=0.000000001)
        except Exception as e:
            pass

In [94]:
import io
import tempfile

def play_speech(text, speaker=1):

    res1 = requests.post("http://localhost:50021/audio_query", params={"text": text, "speaker": speaker})
    data = res1.json()
    display(JSON(data))
    wav_res = requests.post("http://localhost:50021/synthesis", params={"speaker": speaker}, json=data)
    wav_data = wav_res.content
    
    path = tempfile.gettempprefix()+".wav"
    
    with open(path, "wb") as f:
        f.write(wav_data)

    action_map = []
    total_time = 0
    for acc in data["accent_phrases"]:
        for m in acc["moras"]:
            print(m["text"],end="")
            c = m["consonant"]
            if c:
                next_time = total_time + m["consonant_length"]
                action_map.append((total_time, next_time, c))
                total_time = next_time
            v = m["vowel"]
            if v:
                next_time = total_time + m["vowel_length"]
                action_map.append((total_time, next_time, v))
                total_time = next_time
        if acc["pause_mora"]:
            next_time = total_time + acc["pause_mora"]["vowel_length"]
            action_map.append((total_time, next_time, None))
            total_time = next_time
    
    p = subprocess.Popen(["aplay", "-q", path])

    start_time = time.time()
    for a in action_map:
        if a[2]:
            update(a[2], start_time, a[1])
        else:
            time.sleep(a[1] + start_time - time.time())
    p.wait()
    os.remove(path)

In [97]:
play_speech("これは音声発話のテストなんですよ")

<IPython.core.display.JSON object>

コレワオンセエハツワノテストナンデスヨ

In [100]:
play_speech("""
今月17日に予定されていたH3ロケットの初号機打ち上げが中止された異常について、JAXAはメインエンジンに電力を供給するロケットの1段目にある機器で問題が発生したとする調査結果を報告しました。
原因を究明したうえで、来月10日までに再び打ち上げに臨む方針だそうです。報告によれば、メインエンジンの燃焼が始まったあと、燃焼を調整する機器に電力を供給する「VーCON1」と呼ばれる装置の
内部で電流と電圧の値がゼロになる異常が発生していたということです。補助ロケットを含む機体や地上設備のほか、搭載している衛星には損傷がなく、再び打ち上げに向けて対策を講じるとのことです。
ミクの感想：再び打ち上げができるように対策を講じるというのは、良いニュースだね！でも、どんな対策を講じるのか気になるな。安全に打ち上げできるよう、しっかり調査して欲しいでしょうね。""")

<IPython.core.display.JSON object>

コンゲツジュウシチニチニヨテエサレテイタエイチサンロケットノショゴオキウチアゲガチュウシサレタイジョオニツイテジャクサワメインエンジンニデンリョクオキョオキュウスルロケットノイチダンメニアルキキデモンダイガハッセエシタトスルチョオサケッカオホオコクシマシタゲンインオキュウメエシタウエデライゲツトオカマデニフタタビウチアゲニノゾムホオシンダソオデスホオコクニヨレバメインエンジンノネンショオガハジマッタアトネンショオオチョオセエスルキキニデンリョクオキョオキュウスルブイイコンイチトヨバレルソオチノナイブデデンリュウトデンアツノネガゼロニナルイジョオガハッセエシテイタトイウコトデスホジョロケットオフクムキタイヤチジョオセツビノホカトオサイシテイルエエセエニワソンショオガナクフタタビウチアゲニムケテタイサクオコオジルトノコトデスミクノカンソオフタタビウチアゲガデキルヨオニタイサクオコオジルトイウノワヨイニュウスダネデモドンナタイサクオコオジルノカキニナルナアンゼンニウチアゲデキルヨオシッカリチョオサシテホシイデショオネ